In [1]:
import pandas as pd
import numpy as np

In [2]:
BD=pd.read_csv('BD_data.csv')

In [3]:
BD.head()

,BD_ntl,BD_gmap
0,2.99,0
1,2.02,0
2,2.75,0
3,2.90,0
4,1.61,0


In [5]:
BD_ntl=BD.BD_ntl.to_numpy()
BD_ntl=BD_ntl.reshape(-1,1)

In [6]:
from sklearn.mixture import GaussianMixture 
gmm = GaussianMixture(n_components=3,covariance_type='full')
gmm.fit(BD_ntl)

#predictions from gmm
labels = gmm.predict(BD_ntl)
frame = pd.DataFrame(BD_ntl)
frame['cluster'] = labels
frame.columns = ['BD_ntl','cluster']


In [7]:
frame[frame.cluster==0].describe()

,BD_ntl,cluster
count,610728.000000,610728.0
mean,0.443387,0.0
std,0.146395,0.0
min,0.070000,0.0
25%,0.330000,0.0
50%,0.420000,0.0
75%,0.540000,0.0
max,0.800000,0.0


In [8]:
frame[frame.cluster==1].describe()

,BD_ntl,cluster
count,85035.000000,85035.0
mean,1.184120,1.0
std,0.378543,0.0
min,0.810000,1.0
25%,0.900000,1.0
50%,1.050000,1.0
75%,1.360000,1.0
max,2.400000,1.0


In [9]:
frame[frame.cluster==2].describe()

,BD_ntl,cluster
count,17257.000000,17257.0
mean,6.095823,2.0
std,5.950474,0.0
min,2.410000,2.0
25%,2.980000,2.0
50%,4.030000,2.0
75%,6.790000,2.0
max,118.870000,2.0


In [11]:
BD['ntl'] = np.where(BD['BD_ntl']<2.41,'Medium','High')
BD.loc[(BD['BD_ntl'] <0.81),'ntl']='Low'

In [12]:
from scipy.stats import chi2_contingency
from scipy.stats import chi2

In [14]:
confusion_matrix = pd.crosstab(BD["ntl"], BD["BD_gmap"]).as_matrix()

In [15]:
stat, p, dof, expected = chi2_contingency(confusion_matrix)

In [16]:
prob=0.05
alpha = 1.0 - prob
if p <= alpha:
	print('Dependent (reject H0)')
else:
	print('Independent (fail to reject H0)')

Dependent (reject H0)


In [17]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import seaborn as sns



def cramers_v(confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher,
        Journal of the Korean Statistical Society 42 (2013): 323-328
    """
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))
cramers_v(confusion_matrix)

0.33381454038887887